In [1]:
import time
import tensorflow as tf;
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

2022-05-30 16:38:04.747597: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-30 16:38:04.747638: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Chargement de la base de données

In [2]:
print("================ Data Loading ================")
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train = np.reshape(x_train, (-1,28, 28, 1))
x_test = np.reshape(x_test, (-1, 28, 28, 1))

# Data shapes
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)
print("")

================ Data Loading ================
x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)
y_train shape: (60000,)
y_test shape: (10000,)



# Lenet 5

In [3]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=6, kernel_size=(5, 5), activation='relu', input_shape=(28,28,1), trainable=True),
    tf.keras.layers.AveragePooling2D(pool_size=(2,2), strides=2, padding="valid"),
    tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 5), activation='relu', trainable=True),
    tf.keras.layers.AveragePooling2D(pool_size=(2,2), strides=2, padding="valid"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=120, activation='relu',  trainable=True),
    tf.keras.layers.Dense(units=84, activation='relu', trainable=True),
    tf.keras.layers.Dense(units=10, activation='softmax', trainable=True)

]);
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 6)         156       
                                                                 
 average_pooling2d (AverageP  (None, 12, 12, 6)        0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 16)          2416      
                                                                 
 average_pooling2d_1 (Averag  (None, 4, 4, 16)         0         
 ePooling2D)                                                     
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 120)               3

2022-05-30 16:38:08.139346: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-30 16:38:08.139386: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-30 16:38:08.139410: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (big26): /proc/driver/nvidia/version does not exist
2022-05-30 16:38:08.139680: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Objet Quantizer

In [8]:
class Quantizer:
    def __init__(self, model):
        
        # attributs liés au model
        self.model = model
        self.layers = model.layers
        self.weights = []
        self.quantized_weights = None
        self.nb_bits = 8
        
        
    def compile(self,optimizer, loss_fn, metric):
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.acc_metric = metric
        
        
        
    def quantize_w(self, layer):
        if "conv"in layer.name or "dense" in layer.name:
            # getting kernel or weights values
            w = layer.get_weights()[0]
            original_shape = layer.get_weights()[0].shape

            # vectorisation de la matrice de poids
            w = np.reshape(w, (-1))

            # calculer l'espace des valeurs
            a = np.min(w)
            b = np.max(w)
            clamp_w = np.minimum(np.maximum(w, a), b)
            scale = (b - a)/(2**self.nb_bits - 1)

            # calcul des valeurs quantifiées
            q = np.rint(clamp_w/(scale +1e-6)) * scale + a
            q = np.reshape(q,original_shape)

            # Appliquer les nouveaux poids
            layer.set_weights([q, layer.get_weights()[1]])
        
        
    def quantize_activation(self, layer, data):
        activation = layer(data, training = True)
        orig_shape = activation.shape

        # vectorisation
        activation = tf.reshape(activation, (32, -1 ))


        a = tf.math.reduce_min(activation)
        b = tf.math.reduce_max(activation)


        clamp_w = tf.math.minimum(tf.math.maximum(activation, a), b)
        #EMA = (clamp_w * (1/10000) ) + EMA *(1/10000)
        scale = (b - a)/(2**8 - 1)


        # calcul des valeurs quantifiées
        q = tf.math.rint(clamp_w/(scale +1e-6)) * scale + a
        q = tf.reshape(q, orig_shape)
        
        return q
    
    
    def quantized_pred(self, data):
        pass
    
    def train(self,x_train, y_train, epochs = 30, batch_size= 32):
        self.epochs = epochs
        self.batch_size = batch_size
        if x_train.shape[0] % batch_size == 0:
            nb_train_steps = x_train.shape[0] // batch_size
        else:
            nb_train_steps = (x_train.shape[0] // batch_size) + 1
        # Training Loop
        for epoch in range(epochs):
            print(f"Epoch ({epoch +1 }/{epochs})")
            for i in range(nb_train_steps):
                # Batching data
                x = x_train[i*batch_size:(i+1)*batch_size]
                y = y_train[i*batch_size:(i+1)*batch_size]
                
                x = tf.constant(x)
                y = tf.constant(y)
                
                with tf.GradientTape() as tape:
                    # Forward pass
                    for layer in self.layers: 
                        # Quantifier les poids
                        self.quantize_w(layer)
                        # Quantifier les activation
                        x = self.quantize_activation(layer, x)
                       
                    # calcul de la loss
                    loss = self.loss_fn(y, x) 
                # Calcul du gradient
                grads = tape.gradient(loss, self.model.trainable_variables)

                # Decente de gradient
                self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))
                
                # Update training metric.
                self.acc_metric.update_state(y, x)
                
                train_acc = self.acc_metric.result()
                print("  Accuracy: {:.4f} ; loss: {:.4f}".format(float(train_acc),  loss), end='\r')
            print("")


            
    def test(self,data, labels):
        pass
        
        
        
        

In [9]:
Q = Quantizer(model)

Q.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3, momentum = 0.9),
         loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
         metric = tf.keras.metrics.SparseCategoricalAccuracy(),
)
        
Q.train(x_train, y_train)       
        

Epoch (1/30)
  Accuracy: 0.1124 ; loss: 2.3033
Epoch (2/30)
  Accuracy: 0.1124 ; loss: 2.3035
Epoch (3/30)
  Accuracy: 0.1124 ; loss: 2.3037
Epoch (4/30)


KeyboardInterrupt: 